# 4장 좋은 훈련 데이터셋 만들기: 데이터 전처리

## 4-1

## 누락된 데이터 다루기

### 테이블 형태 데이터에서 누락된 값 식별

In [1]:
import pandas as pd
from io import StringIO  # 문자열을 파일처럼(csv) 사용하게 해주는 클래스
import sys

csv_data = \  # 열 : 특성 ; 행 : 샘플
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


NaN : Not a Number; 결측치(누락된 데이터)

In [2]:
# pd.isnull() : null 값인지를 0/1로 반환
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64


- pd.isna() : isnull과 동일
- axis = 0 -> 열 기준 탐색
- axis = 1 -> 행 기준 탐색

### 누락된 값이 있는 훈련 샘플이나 특성 제외 : pd.dropna()

In [12]:
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [10]:
# pd.dropna() : 누락된 값이 있는 행을 삭제합니다
df.dropna()

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [11]:
# axis=1 : 누락된 값이 있는 열을 삭제합니다
df.dropna(axis=1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [4]:
# how='all' : 모든 열이 NaN인 행을 삭제합니다
df.dropna(how='all')  

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [13]:
# thresh=n : NaN이 아닌 값이 n개보다 작은 행을 삭제합니다
df.dropna(thresh=4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [6]:
# subset=['열이름'] : 특정 열에 NaN이 있는 행만 삭제합니다
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


### 누락된 값 대체 : sklearn.impute - SimpleImputer

In [7]:
# 열의 평균으로 누락된 값 대체하기

from sklearn.impute import SimpleImputer
import numpy as np

# missing_values=np.nan : NaN을 누락된 값으로 설정(기본값)
# strategy='mean' : 누락된 값을 평균값으로 대체
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

7.5 == 3과 12의 평균값

6 == 4와 8의 평균값

- strategy = 'median' : 중간값
- strategy = 'most_frequent' : 최빈값
- strategy = 'constant' : constant 자리에 상수값 입력

In [8]:
# SimpleImputer와 FunctionTransformer를 사용하여 행의 평균 계산

from sklearn.preprocessing import FunctionTransformer

ftr_imr = FunctionTransformer(lambda X: imr.fit_transform(X.T).T)
imputed_data = ftr_imr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])

6.333 == 5, 6, 8의 평균

11 == 10, 11, 12의 평균

In [14]:
# add_indicator=True :
# indicator_ 속성 추가; 누락된 값의 위치를 포함한 배열 반환
imr = SimpleImputer(add_indicator=True)
imputed_data = imr.fit_transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ,  0. ,  0. ],
       [ 5. ,  6. ,  7.5,  8. ,  1. ,  0. ],
       [10. , 11. , 12. ,  6. ,  0. ,  1. ]])

- 뒤에서 두 열은 기존 데이터에서 결측치가 있는 열(기존 데이터에서 뒤 두 열)의 결측치 위치를 의미
- indicator_ 속성은 MissingIndicator 클래스의 객체

In [15]:
# MissingIndicator.features_ 속성 : 누락된 값이 있는 특성의 인덱스
imr.indicator_.features_

array([2, 3], dtype=int64)

In [16]:
# MissingIndicator.fit_transform() :
# features_에서 누락된 값의 위치를 나타내는 배열 반환
imr.indicator_.fit_transform(df.values)

array([[False, False],
       [ True, False],
       [False,  True]])

- ```SimpleImputer```는 한 특성의 통곗값을 사용하여 누락된 값을 채웁니다.

- ```IterativeImputer``` 클래스는 다른 특성을 사용하여 누락된 값을 예측합니다. ```initial_strategy``` 매개변수에 지정된 방식으로 누락된 값을 초기화합니다. 그다음 누락된 값이 있는 한 특성을 타깃으로 삼고 다른 특성을 사용해 모델을 훈련하여 예측합니다. 이런 식으로 누락된 값이 있는 모든 특성을 순회합니다.

- ```initial_strategy``` 매개변수에 지정할 수 있는 값은 ```SimpleImputer```와 동일하게 ```'mean'```, ```'median'```, ```'most_frequent'```, ```'constant'```가 가능합니다.


<예측할 특성을 선택하는 순서>

- ```'ascending'``` : 누락된 값이 가장 적은 특성부터 선택 <기본값>
- ```'descending'``` : 누락된 값이 가장 큰 특성부터 선택
- ```'roman'``` : 왼쪽에서 오른쪽으로 선택
- ```'arabic'``` : 오른쪽에서 왼쪽으로 선택
- ```'random'``` : 랜덤하게 선택

특성 예측은 종료 조건을 만족할 때까지 반복합니다. 각 반복 단계에서 이전 단계와 절댓값 차이 중 가장 큰 값이 누락된 값을 제외하고 가장 큰 절댓값에 ```tol``` 매개변수를 곱한 것보다 작을 경우 종료합니다. ```tol``` 매개변수 기본값은 1e-3입니다. 또는 ```max_iter``` 매개변수에서 지정한 횟수에 도달할 때 종료합니다. ```max_iter```의 기본값은 10입니다.

예측에 사용하는 모델은 ```estimator``` 매개변수에서 지정할 수 있으며 기본적으로 ```BayesianRidge``` 클래스를 사용합니다. 예측에 사용할 특성 개수는 ```n_nearest_features```에서 지정할 수 있으며 상관 계수가 높은 특성을 우선하여 랜덤하게 선택합니다. 기본값은 None으로 모든 특성을 사용합니다.

In [17]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

iimr = IterativeImputer()
iimr.fit_transform(df.values)

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  7.00047063,  8.        ],
       [10.        , 11.        , 12.        , 12.99964527]])

In [18]:
# KNNImputer : K-최근접 이웃 방법을 사용해 결측치 채움
# n_neighbors : 근접 이웃 개수 매개변수, 기본값 5
# n_neighbors보다 샘플의 개수가 작으면 mean 적용

from sklearn.impute import KNNImputer

kimr = KNNImputer()
kimr.fit_transform(df.values)

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [19]:
# pd.fillna() : 결측치를 매개변수 값으로 채움

df.fillna(df.mean())

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,10.0,11.0,12.0,6.0


In [20]:
# method = 'bfill' : 누락된 값을 다음 행의 값으로 채움

df.fillna(method='bfill') # method='backfill'와 같습니다

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,12.0,8.0
2,10.0,11.0,12.0,NaN


In [21]:
# method = 'ffill' : 누락된 값을 이전 행의 값으로 채움

df.fillna(method='ffill') # method='pad'와 같습니다

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,3.0,8.0
2,10.0,11.0,12.0,8.0


In [22]:
# axis=1 : 행X 열 사용

df.fillna(method='ffill', axis=1)

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,6.0,8.0
2,10.0,11.0,12.0,12.0


## 범주형 데이터 다루기

### 판다스를 사용한 범주형 데이터 인코딩

In [23]:
import pandas as pd

df = pd.DataFrame([['green', 'M', 10.1, 'class2'],
                   ['red', 'L', 13.5, 'class1'],
                   ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


class1, 2 : target

### 순서가 있는 특성 매핑

In [24]:
size_mapping = {'XL': 3,
                'L': 2,
                'M': 1}

# pd.map() : 매핑 메서드
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [25]:
# 딕셔너리를 사용하여 매핑 정보 생성
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

### 클래스 레이블 인코딩

In [32]:
import numpy as np

# 클래스 레이블을 문자열에서 정수로 바꾸기 위해 매핑 딕셔너리 생성
class_mapping = {label: idx
                 for idx, label
                 in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [27]:
# pd.map() 사용하여 클래스 레이블을 문자열에서 정수로 변환
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [33]:
# v: k for k, v -> 클래스 레이블을 거꾸로 매핑
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,NaN
1,red,2,13.5,NaN
2,blue,3,15.3,NaN


In [29]:
# 사이킷런의 LabelEncoder을 사용한 레이블 인코딩

from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1])

In [30]:
# inverse_transform : 거꾸로 매핑
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

In [31]:
class_le.classes_

array(['class1', 'class2'], dtype=object)